In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import pandas as pd
import warnings
import sys
import numpy as np
import scipy as sc
import random
from scipy import stats
from numpy import log10, sqrt

mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def def_display_df_misses(df):
    df = df[df['Payment Reduction Reproduced?'] == 0]
    items = ['file_year', 'HAI Measures End Date', 'HAI Measures Start Date',
             'CAUTI Footnote', 'CAUTI W Z Score', 'CAUTI SIR W Z Score',
             'CDI Footnote', 'CDI W Z Score', 'CDI SIR W Z Score',
             'CLABSI Footnote', 'CLABSI W Z Score', 'CLABSI SIR W Z Score',
             'MRSA Footnote', 'MRSA W Z Score', 'MRSA SIR W Z Score',
             'SSI Footnote', 'SSI W Z Score', 'SSI SIR W Z Score',
             'PSI-90 Footnote', 'PSI-90 W Z Score', 'PSI-90 SIR W Z Score',
             'Total HAC Footnote', 'Total HAC Score', 'Total HAC Score (derived)',
             'Payment Reduction Footnote', 'Payment Reduction', 
             'Payment Reduction (derived)', 'Payment Reduction Reproduced?',
             ]
    return df.filter(items=items)


def Winsorize_it(x, WinZs):
    
    x2 = []
    for i, val in enumerate(x):
        wz = float(WinZs[i])
        val = float(val)
        if np.isnan(wz) == True:
            x2.append(np.nan)
        else:
            x2.append(val)
    
    p5 = np.nanpercentile(x2, 5)
    p95 = np.nanpercentile(x2, 95)
    WinScores = []
    
    for i, val in enumerate(x2):
        wz = float(WinZs[i])
        val = float(val)
        
        if np.isnan(wz) == True:
            WinScores.append(np.nan)
            
        elif np.isnan(wz) == False:
            if val >= p5 and val <= p95:
                WinScores.append(val)
            elif val < p5:
                WinScores.append(p5)
            elif val > p95:
                WinScores.append(p95)
            elif np.isnan(val) == True:
                #print('val:', val, '|', WinZs[i])
                WinScores.append(np.nan)
        
    
    return WinScores
        

def ZScore_it(x, WinZs):
    
    x2 = []
    for i, val in enumerate(x):
        wz = float(WinZs[i])
        val = float(val)
        if np.isnan(wz) == True:
            x2.append(np.nan)
        else:
            x2.append(val)
    
    x2 = np.array(x2)
    avg = np.nanmean(x2)
    std = np.nanstd(x2)
    zscores = (x2 - avg) / std
    return zscores


## Load HAC file

In [2]:
hac_df = pd.read_pickle(mydir + "data/CareCompare_data/CombinedFiles_HACRP/Facility.pkl")
hac_df = hac_df[hac_df['file_year'] == '2019']
hac_df = hac_df[hac_df['file_month'] == '10']

print('HACRP years:', sorted(hac_df['Fiscal Year'].unique()))

features = ['CAUTI Score', 'CLABSI Score', 'CDI Score', 'MRSA Score', 'SSI Score',
            'Total HAC Score', 'Domain 1 Score', 'AHRQ PSI-90 Score', 'Domain 2 Score',
            'CAUTI W Z Score', 'CLABSI W Z Score', 'MRSA W Z Score', 'CDI W Z Score', 
            'SSI W Z Score', 'PSI-90 W Z Score']
for f in features:
    hac_df[f] = hac_df[f].astype(str)
    hac_df[f] = hac_df[f].str.replace('*', '')
    hac_df[f] = pd.to_numeric(hac_df[f], errors='coerce')

hac_df.dropna(how='all', axis=1, inplace=True)
hac_df.head()

HACRP years: [2019]


,CAUTI Footnote,CAUTI W Z Score,CDI Footnote,CDI W Z Score,CLABSI Footnote,CLABSI W Z Score,Domain 1 End Date,Domain 1 Footnote,Domain 1 Score,Domain 1 Start Date,Domain 2 End Date,Domain 2 Footnote,Domain 2 Score,Domain 2 Start Date,Facility ID,Facility Name,Fiscal Year,MRSA Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI W Z Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year
0,NaN,0.2434,NaN,-0.8858,NaN,0.4992,06/30/2017,NaN,-0.6505,10/01/2015,12/31/2017,NaN,-0.1112,01/01/2016,010001,SOUTHEAST ALABAMA MEDICAL CENTER,2019,NaN,0.5608,NaN,-0.6505,No,NaN,NaN,-0.9737,AL,NaN,-0.1921,10,2019
1,NaN,0.0201,NaN,-1.0295,NaN,-0.3761,06/30/2017,NaN,1.0790,10/01/2015,12/31/2017,NaN,-0.1370,01/01/2016,010005,MARSHALL MEDICAL CENTER SOUTH,2019,NaN,0.8634,NaN,1.0790,No,NaN,NaN,-0.1626,AL,NaN,0.0454,10,2019
2,NaN,0.6987,NaN,-0.9273,NaN,0.4309,06/30/2017,NaN,-0.1718,10/01/2015,12/31/2017,NaN,0.1553,01/01/2016,010006,ELIZA COFFEE MEMORIAL HOSPITAL,2019,NaN,1.1477,NaN,-0.1718,No,NaN,NaN,-0.5737,AL,NaN,0.1062,10,2019
3,NaN,-1.5202,NaN,-1.9247,5.0,NaN,06/30/2017,NaN,0.4116,10/01/2015,12/31/2017,NaN,-1.7225,01/01/2016,010007,MIZELL MEMORIAL HOSPITAL,2019,5.0,NaN,NaN,0.4116,No,NaN,5.0,NaN,AL,NaN,-1.4024,10,2019
4,5.0,NaN,5.0,NaN,5.0,NaN,06/30/2017,NaN,-0.0101,10/01/2015,12/31/2017,5.0,NaN,01/01/2016,010008,CRENSHAW COMMUNITY HOSPITAL,2019,5.0,NaN,NaN,-0.0101,No,NaN,5.0,NaN,AL,NaN,-0.0101,10,2019


## Format dates within the HAC file 

In [3]:

hac_df['Domain 2 Start Date'] = pd.to_datetime(hac_df['Domain 2 Start Date'])
hac_df.style.format({'Domain 2 Start Date': lambda t: t.strftime("%Y-%m-%d")})

hac_df['Domain 2 End Date'] = pd.to_datetime(hac_df['Domain 2 End Date'])
hac_df.style.format({'Domain 2 End Date': lambda t: t.strftime("%Y-%m-%d")})

hac_df.rename(columns={'Domain 2 Start Date': 'Start Date', 'Domain 2 End Date': 'End Date'}, inplace=True)

print('rows in hac_df, after reformatting dates:', hac_df.shape[0])
print(hac_df['Start Date'].unique())
print(hac_df['End Date'].unique())


rows in hac_df, after reformatting dates: 3281
['2016-01-01T00:00:00.000000000']
['2017-12-31T00:00:00.000000000']


## Load and merge HAI files

In [4]:
##############################   CAUTI   ################################################

cauti_df = pd.read_pickle(mydir + "data/preprocessed_HAI_data/CAUTI_Data.pkl")
cauti_df = cauti_df.filter(items=['Facility ID', 'CAUTI Urinary Catheter Days (ICUs + select wards)', 
                                  'CAUTI Observed Cases (ICUs + select wards)', 
                                  'CAUTI Predicted Cases (ICUs + select wards)', 
                                  'Start Date', 'End Date'], axis=1)

cauti_df.rename(columns={'CAUTI Urinary Catheter Days (ICUs + select wards)': 'CAUTI Urinary Catheter Days', 
                         'CAUTI Observed Cases (ICUs + select wards)': 'CAUTI Observed Cases', 
                         'CAUTI Predicted Cases (ICUs + select wards)': 'CAUTI Predicted Cases', 
                        }, inplace=True)

features = ['CAUTI Urinary Catheter Days', 'CAUTI Observed Cases', 'CAUTI Predicted Cases']
for f in features:
    cauti_df[f] = cauti_df[f].astype(str)
    cauti_df[f] = pd.to_numeric(cauti_df[f], errors='coerce')

cauti_df['Start Date'] = pd.to_datetime(cauti_df['Start Date'])
cauti_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
cauti_df['End Date'] = pd.to_datetime(cauti_df['End Date'])
cauti_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})
cauti_df = cauti_df[(cauti_df['Start Date'].isin(hac_df['Start Date'].unique())) | (cauti_df['End Date'].isin(hac_df['End Date'].unique()))]

cauti_df.head()


,Facility ID,CAUTI Urinary Catheter Days,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date
20,450651,14923.0,20.0,31.259,2016-01-01,2016-12-31
35,050230,2520.0,1.0,1.766,2016-01-01,2016-12-31
48,670102,NaN,NaN,NaN,2017-01-01,2017-12-31
56,330004,8284.0,1.0,5.768,2016-01-01,2016-12-31
90,230301,8.0,0.0,0.004,2017-01-01,2017-12-31


In [5]:
##############################   CLABSI   ###############################################

clabsi_df = pd.read_pickle(mydir + "data/preprocessed_HAI_data/CLABSI_Data.pkl")
clabsi_df = clabsi_df.filter(items=['Facility ID', 'CLABSI Device Days (ICUs + select wards)', 
                                    'CLABSI Observed Cases (ICUs + select wards)', 
                                    'CLABSI Predicted Cases (ICUs + select wards)', 
                                    'Start Date', 'End Date'], axis=1)

clabsi_df.rename(columns={'CLABSI Device Days (ICUs + select wards)': 'CLABSI Device Days', 
                          'CLABSI Observed Cases (ICUs + select wards)': 'CLABSI Observed Cases', 
                          'CLABSI Predicted Cases (ICUs + select wards)': 'CLABSI Predicted Cases', 
                        }, inplace=True)


features = ['CLABSI Device Days', 'CLABSI Observed Cases', 'CLABSI Predicted Cases']
for f in features:
    clabsi_df[f] = clabsi_df[f].astype(str)
    clabsi_df[f] = pd.to_numeric(clabsi_df[f], errors='coerce')

clabsi_df['Start Date'] = pd.to_datetime(clabsi_df['Start Date'])
clabsi_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
clabsi_df['End Date'] = pd.to_datetime(clabsi_df['End Date'])
clabsi_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})
clabsi_df = clabsi_df[(clabsi_df['Start Date'].isin(hac_df['Start Date'].unique())) | (clabsi_df['End Date'].isin(hac_df['End Date'].unique()))]

clabsi_df.head()

,Facility ID,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,Start Date,End Date
1,390008,297.0,0.0,0.174,2017-01-01,2017-12-31
14,110111,387.0,0.0,0.225,2016-01-01,2016-12-31
48,170110,194.0,0.0,0.112,2016-01-01,2016-12-31
78,310130,1041.0,0.0,0.733,2017-01-01,2017-12-31
85,170049,6998.0,1.0,5.587,2017-01-01,2017-12-31


In [6]:

##############################   MRSA   ###############################################

mrsa_df = pd.read_pickle(mydir + "data/preprocessed_HAI_data/MRSA_Data.pkl")
mrsa_df = mrsa_df.filter(items=['Facility ID', 'MRSA patient days', 
                                'MRSA Observed Cases', 'MRSA Predicted Cases', 
                                'Start Date', 'End Date'], axis=1)
mrsa_df['Start Date'] = pd.to_datetime(mrsa_df['Start Date'])
mrsa_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
mrsa_df['End Date'] = pd.to_datetime(mrsa_df['End Date'])
mrsa_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})

mrsa_df = mrsa_df[(mrsa_df['Start Date'].isin(hac_df['Start Date'].unique())) | (mrsa_df['End Date'].isin(hac_df['End Date'].unique()))]

##############################   CDI   ###############################################

cdi_df = pd.read_pickle(mydir + "data/preprocessed_HAI_data/CDI_Data.pkl")
cdi_df = cdi_df.filter(items=['Facility ID', 'CDIFF patient days', 
                              'CDIFF Observed Cases', 'CDIFF Predicted Cases', 
                              'Start Date', 'End Date'], axis=1)
cdi_df['Start Date'] = pd.to_datetime(cdi_df['Start Date'])
cdi_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
cdi_df['End Date'] = pd.to_datetime(cdi_df['End Date'])
cdi_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})

cdi_df = cdi_df[(cdi_df['Start Date'].isin(hac_df['Start Date'].unique())) | (cdi_df['End Date'].isin(hac_df['End Date'].unique()))]

###################  Merge CAUTI, CLABSI, MRSA, and CDI  ####################################

hai_df = cauti_df.merge(clabsi_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')
hai_df = hai_df.merge(mrsa_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')
hai_df = hai_df.merge(cdi_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')


####################  Drop duplicate rows resulting from merger #############################

hai_df.drop_duplicates(inplace=True)


########################  Conversions to numeric  #############################################

features = ['CAUTI Urinary Catheter Days', 'CLABSI Device Days',
            'MRSA patient days', 'CDIFF patient days',
            'CLABSI Observed Cases', 'CLABSI Predicted Cases', 
            'MRSA Observed Cases', 'MRSA Predicted Cases', 
            'CDIFF Observed Cases', 'CDIFF Predicted Cases', 
            'CAUTI Observed Cases', 'CAUTI Predicted Cases', 
           ]

for f in features:
    hai_df[f] = hai_df[f].astype(str)
    hai_df[f] = hai_df[f].str.replace('*', '')
    hai_df[f] = hai_df[f].str.replace(' ', '')
    hai_df[f] = pd.to_numeric(hai_df[f], errors='coerce')
    
hai_df['Total device days'] = hai_df['CLABSI Device Days'] + hai_df['CAUTI Urinary Catheter Days']


########################  Reorder columns  #############################################

col_to_move = hai_df.pop('CAUTI Urinary Catheter Days')
hai_df.insert(hai_df.shape[1] - 2, 'CAUTI Urinary Catheter Days', col_to_move)

print(hai_df.shape)
hai_df.head()

(9605, 16)


,Facility ID,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,MRSA patient days,MRSA Observed Cases,MRSA Predicted Cases,CDIFF patient days,CDIFF Observed Cases,CAUTI Urinary Catheter Days,CDIFF Predicted Cases,Total device days
0,450651,20.0,31.259,2016-01-01,2016-12-31,15342.0,15.0,17.420,117372.0,5.0,6.179,103727.0,101.0,14923.0,78.891,30265.0
1,050230,1.0,1.766,2016-01-01,2016-12-31,1681.0,1.0,1.194,20334.0,4.0,0.853,19088.0,14.0,2520.0,9.453,4201.0
2,670102,NaN,NaN,2017-01-01,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,330004,1.0,5.768,2016-01-01,2016-12-31,5157.0,0.0,3.481,32866.0,3.0,1.265,32866.0,21.0,8284.0,17.573,13441.0
4,230301,0.0,0.004,2017-01-01,2017-12-31,NaN,NaN,NaN,1483.0,0.0,0.028,1483.0,0.0,8.0,0.323,NaN


## Filter HAI data on start dates and end dates that match those in the HAC file.

In [7]:
#########  Filter on start dates and end dates that match those in the HAC file  #######

hai_df = hai_df[(hai_df['Start Date'].isin(['2016-01-01', '2017-01-01'])) & (hai_df['End Date'].isin(['2016-12-31', '2017-12-31']))]

print(hai_df.shape)
print(hai_df['Start Date'].unique())
print(hai_df['End Date'].unique())

(9605, 16)
['2016-01-01T00:00:00.000000000' '2017-01-01T00:00:00.000000000']
['2016-12-31T00:00:00.000000000' '2017-12-31T00:00:00.000000000']


## Drop hospitals from HAI data that are not contained in HAC data.

In [8]:
# drop hospitals in HAI data that are not in HAC data
hai_df = hai_df[hai_df['Facility ID'].isin(hac_df['Facility ID'].unique())]

In [9]:
# hospitals in the hac data but not in the hai data ...
#tdf = hac_df.copy(deep=True) 
tdf = hac_df[~hac_df['Facility ID'].isin(hai_df['Facility ID'].unique())]
hac_df = hac_df[hac_df['Facility ID'].isin(hai_df['Facility ID'].unique())]

print(len(tdf['Facility ID'].unique()), 'hospitals in HAC dataset but not in HAI dataset\n')


tdf.head()

7 hospitals in HAC dataset but not in HAI dataset



,CAUTI Footnote,CAUTI W Z Score,CDI Footnote,CDI W Z Score,CLABSI Footnote,CLABSI W Z Score,Domain 1 End Date,Domain 1 Footnote,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Footnote,Domain 2 Score,Start Date,Facility ID,Facility Name,Fiscal Year,MRSA Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI W Z Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year
540,5.0,NaN,5.0,NaN,5.0,NaN,06/30/2017,5.0,NaN,10/01/2015,2017-12-31,5.0,NaN,2016-01-01,060129,UCHEALTH BROOMFIELD HOSPITAL LLC,2019,5.0,NaN,5.0,NaN,No,NaN,5.0,NaN,CO,5.0,NaN,10,2019
541,5.0,NaN,5.0,NaN,5.0,NaN,06/30/2017,5.0,NaN,10/01/2015,2017-12-31,5.0,NaN,2016-01-01,060130,UCHEALTH GRANDVIEW HOSPITAL,2019,5.0,NaN,5.0,NaN,No,NaN,5.0,NaN,CO,5.0,NaN,10,2019
880,5.0,NaN,5.0,NaN,5.0,NaN,06/30/2017,5.0,NaN,10/01/2015,2017-12-31,5.0,NaN,2016-01-01,130072,SAINT ALPHONSUS NEIGHBORHOOD HOSPITAL OF SOUNT...,2019,5.0,NaN,5.0,NaN,No,NaN,5.0,NaN,ID,5.0,NaN,10,2019
1786,5.0,NaN,5.0,NaN,5.0,NaN,06/30/2017,5.0,NaN,10/01/2015,2017-12-31,5.0,NaN,2016-01-01,290058,DIGNITY HEALTH ST ROSE DOMINICAN-NORTH LAS VEG...,2019,5.0,NaN,5.0,NaN,No,NaN,5.0,NaN,NV,5.0,NaN,10,2019
2350,5.0,NaN,5.0,NaN,5.0,NaN,06/30/2017,5.0,NaN,10/01/2015,2017-12-31,5.0,NaN,2016-01-01,370239,SAYRE COMMUNITY HOSPITAL,2019,5.0,NaN,5.0,NaN,No,NaN,5.0,NaN,OK,5.0,NaN,10,2019


## Correct HAI file for non-duplicate rows having duplicate dates

**Problem:** Some rows for the same provider have duplicate measurement dates but different values for observed cases, predicted cases, etc. This results from each year having multiple (quarterly) files, the data within which can vary among files. Additionally, the most recent file for each year is not always the right file to use. 

**Need:** Since only one row can be used, we need to figure out which row should be used.

**Solution:** Select the row with the greatest totals for predicted cases for each HAI.

In [10]:
##############  Label rows that have duplicate dates (per provider) ####################
##############  For each provider with rows having duplicate dates,  ###################
###########  keep the last row (will have greatest number of total device days)  #######

hai_df['duplicated dates'] = hai_df.duplicated(subset=['Facility ID', 'Start Date', 'End Date'], keep=False)

hai_df.sort_values(by=['Facility ID', 'Start Date', 'End Date', 
                       'MRSA Predicted Cases', 'CAUTI Predicted Cases',  
                       'MRSA Observed Cases', 'CAUTI Observed Cases', 
                       'CDIFF Predicted Cases', 'CLABSI Predicted Cases',
                       'CDIFF Observed Cases', 'CLABSI Observed Cases',
                       ], inplace=True, ascending=False)

hai_df.drop_duplicates(subset=['Facility ID', 'Start Date', 'End Date'], inplace=True, keep='first')

print(hai_df.shape)
hai_df.head()

(6538, 17)


,Facility ID,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,MRSA patient days,MRSA Observed Cases,MRSA Predicted Cases,CDIFF patient days,CDIFF Observed Cases,CAUTI Urinary Catheter Days,CDIFF Predicted Cases,Total device days,duplicated dates
4808,670124,0.0,0.003,2017-01-01,2017-12-31,NaN,NaN,NaN,62.0,0.0,0.001,62.0,0.0,5.0,0.011,NaN,False
4859,670122,0.0,0.862,2017-01-01,2017-12-31,861.0,0.0,0.602,10424.0,0.0,0.455,8311.0,5.0,1258.0,5.016,2119.0,False
1446,670121,NaN,NaN,2017-01-01,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1062,670120,0.0,0.758,2017-01-01,2017-12-31,620.0,0.0,0.475,8715.0,0.0,0.253,8715.0,4.0,1026.0,3.281,1646.0,False
2192,670120,NaN,NaN,2016-01-01,2016-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


## Aggregate annual HAI data into biennial data

Purpose: Match the biennial measurement periods of HAC data


In [11]:
start_dates = hac_df['Start Date'].tolist()
end_dates = hac_df['End Date'].tolist()
prvdrs = hac_df['Facility ID'].tolist()

total_device_days = []

cauti_days = []
clabsi_days = []
mrsa_days = []
cdi_days = []

cauti_pred = []
clabsi_pred = []
mrsa_pred = []
cdi_pred = []

cauti_obs = []
clabsi_obs = []
mrsa_obs = []
cdi_obs = []

for i, start in enumerate(start_dates):
    end = end_dates[i]
    prvdr = prvdrs[i]
    
    tdf = hai_df[hai_df['Facility ID'] == prvdr]
    tdf = tdf[(tdf['Start Date'] == start) | (tdf['End Date'] == end)]
    
    if tdf.shape[0] == 1:
        pass
        #print('tdf.shape[0]:', 1)
        #print('hospital:', prvdr)
        #print(tdf['Start Date'].unique())
        #print(tdf['End Date'].unique())
        
    if tdf.shape[0] > 2:
        print('tdf.shape[0] = ', tdf.shape[0])
        for date_ in ['Start Date', 'End Date']:
            tdf['duplicated dates'] = tdf.duplicated(subset=[date_], keep=False)
            tdf.sort_values(by=[
                                'CAUTI Observed Cases', 'CLABSI Observed Cases',
                                'MRSA Observed Cases', 'CDIFF Observed Cases',
                                ], inplace=True, ascending=True)

            tdf.drop_duplicates(subset=[date_], inplace=True, keep='last')

        if tdf.shape[0] > 2:
            print('Error:')
            print("tdf.shape[0] > 2:", tdf.shape[0])
            print(start)
            print(end)
            print(tdf.head())
            break
        
    total_device_days.append(np.nansum(tdf['Total device days']))
    cauti_days.append(np.nansum(tdf['CAUTI Urinary Catheter Days']))
    clabsi_days.append(np.nansum(tdf['CLABSI Device Days']))
    mrsa_days.append(np.nansum(tdf['MRSA patient days']))
    cdi_days.append(np.nansum(tdf['CDIFF patient days']))
        
    cauti_pred.append(np.nansum(tdf['CAUTI Predicted Cases']))
    clabsi_pred.append(np.nansum(tdf['CLABSI Predicted Cases']))
    mrsa_pred.append(np.nansum(tdf['MRSA Predicted Cases']))
    cdi_pred.append(np.nansum(tdf['CDIFF Predicted Cases']))

    cauti_obs.append(np.nansum(tdf['CAUTI Observed Cases']))
    clabsi_obs.append(np.nansum(tdf['CLABSI Observed Cases']))
    mrsa_obs.append(np.nansum(tdf['MRSA Observed Cases']))
    cdi_obs.append(np.nansum(tdf['CDIFF Observed Cases']))
    

## Add HAI data to the HAC dataframe and save

In [12]:
hac_df['Total device days'] = total_device_days
hac_df['CAUTI Urinary Catheter Days'] = cauti_days
hac_df['CLABSI Device Days'] = clabsi_days
hac_df['MRSA patient days'] = mrsa_days
hac_df['CDI patient days'] = cdi_days

hac_df['CAUTI Observed Cases'] = cauti_obs
hac_df['CLABSI Observed Cases'] = clabsi_obs
hac_df['MRSA Observed Cases'] = mrsa_obs
hac_df['CDI Observed Cases'] = cdi_obs

hac_df['CAUTI Predicted Cases'] = cauti_pred
hac_df['CLABSI Predicted Cases'] = clabsi_pred
hac_df['MRSA Predicted Cases'] = mrsa_pred
hac_df['CDI Predicted Cases'] = cdi_pred

hac_df['CAUTI derived SIR'] = np.round(hac_df['CAUTI Observed Cases'] / hac_df['CAUTI Predicted Cases'],4)
hac_df['CLABSI derived SIR'] = np.round(hac_df['CLABSI Observed Cases'] / hac_df['CLABSI Predicted Cases'],4)
hac_df['MRSA derived SIR'] = np.round(hac_df['MRSA Observed Cases'] / hac_df['MRSA Predicted Cases'],4)
hac_df['CDI derived SIR'] = np.round(hac_df['CDI Observed Cases'] / hac_df['CDI Predicted Cases'],4)

print('hac_df.shape:', hac_df.shape)
print(len(hac_df['Facility ID'].unique()), 'hospitals in 2019 HACRP')
hac_df.head()



hac_df.shape: (3274, 47)
3274 hospitals in 2019 HACRP


,CAUTI Footnote,CAUTI W Z Score,CDI Footnote,CDI W Z Score,CLABSI Footnote,CLABSI W Z Score,Domain 1 End Date,Domain 1 Footnote,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Footnote,Domain 2 Score,Start Date,Facility ID,Facility Name,Fiscal Year,MRSA Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI W Z Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,MRSA patient days,CDI patient days,CAUTI Observed Cases,CLABSI Observed Cases,MRSA Observed Cases,CDI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,MRSA Predicted Cases,CDI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,MRSA derived SIR,CDI derived SIR
0,NaN,0.2434,NaN,-0.8858,NaN,0.4992,06/30/2017,NaN,-0.6505,10/01/2015,2017-12-31,NaN,-0.1112,2016-01-01,010001,SOUTHEAST ALABAMA MEDICAL CENTER,2019,NaN,0.5608,NaN,-0.6505,No,NaN,NaN,-0.9737,AL,NaN,-0.1921,10,2019,41414.0,26287.0,15127.0,198179.0,196893.0,25.0,13.0,13.0,71.0,24.735,12.077,10.860,144.590,1.0107,1.0764,1.1971,0.4910
1,NaN,0.0201,NaN,-1.0295,NaN,-0.3761,06/30/2017,NaN,1.0790,10/01/2015,2017-12-31,NaN,-0.1370,2016-01-01,010005,MARSHALL MEDICAL CENTER SOUTH,2019,NaN,0.8634,NaN,1.0790,No,NaN,NaN,-0.1626,AL,NaN,0.0454,10,2019,18576.0,13423.0,5153.0,82946.0,78581.0,7.0,2.0,4.0,11.0,7.926,3.324,2.903,25.083,0.8832,0.6017,1.3779,0.4385
2,NaN,0.6987,NaN,-0.9273,NaN,0.4309,06/30/2017,NaN,-0.1718,10/01/2015,2017-12-31,NaN,0.1553,2016-01-01,010006,ELIZA COFFEE MEMORIAL HOSPITAL,2019,NaN,1.1477,NaN,-0.1718,No,NaN,NaN,-0.5737,AL,NaN,0.1062,10,2019,37615.0,22149.0,15466.0,127185.0,121189.0,26.0,13.0,11.0,32.0,20.443,12.509,7.108,67.261,1.2718,1.0393,1.5476,0.4758
3,NaN,-1.5202,NaN,-1.9247,5.0,NaN,06/30/2017,NaN,0.4116,10/01/2015,2017-12-31,NaN,-1.7225,2016-01-01,010007,MIZELL MEMORIAL HOSPITAL,2019,5.0,NaN,NaN,0.4116,No,NaN,5.0,NaN,AL,NaN,-1.4024,10,2019,2548.0,2031.0,517.0,6589.0,5746.0,0.0,0.0,0.0,0.0,1.158,0.314,0.137,3.067,0.0000,0.0000,0.0000,0.0000
4,5.0,NaN,5.0,NaN,5.0,NaN,06/30/2017,NaN,-0.0101,10/01/2015,2017-12-31,5.0,NaN,2016-01-01,010008,CRENSHAW COMMUNITY HOSPITAL,2019,5.0,NaN,NaN,-0.0101,No,NaN,5.0,NaN,AL,NaN,-0.0101,10,2019,644.0,554.0,90.0,0.0,6.0,0.0,0.0,0.0,1.0,0.301,0.052,0.000,0.001,0.0000,0.0000,NaN,1000.0000


## Generate Winsorized z-scores

In [13]:
hais = ['CAUTI', 'CLABSI', 'MRSA', 'CDI']

df_2019 = hac_df.copy(deep=True)
    
for i, hai in enumerate(hais):
    tdf2 = df_2019[~df_2019[hai + ' Footnote'].isin([18, '18', '18 ', ' 18', 
                                              5, '5', ' 5', '5 ',
                                              4, '4', ' 4', '4 ',
                                              ])]
      
    reported_winZ = tdf2[hai + ' W Z Score'].tolist()
    sirs = tdf2[hai + ' derived SIR'].tolist()
    tdf2[hai + ' derived Winsorized SIR'] = Winsorize_it(sirs, reported_winZ)
    tdf2[hai + ' derived W Z Score'] = ZScore_it(tdf2[hai + ' derived Winsorized SIR'], reported_winZ)
    
    # Assign maximum WinZ scores to hospitals with HAI footnote 18 
    maxWinZ = np.nanmax(tdf2[hai + ' derived W Z Score'])
    tdf3 = df_2019[df_2019[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    tdf3[hai + ' derived Winsorized SIR'] = [np.nan]*tdf3.shape[0]
    tdf3[hai + ' derived W Z Score'] = [maxWinZ]*tdf3.shape[0]
    
    tdf4 = df_2019[df_2019[hai + ' Footnote'].isin([5, '5', ' 5', '5 ', 4, '4', ' 4', '4 '])]
    tdf4[hai + ' derived Winsorized SIR'] = [np.nan]*tdf4.shape[0]
    tdf4[hai + ' derived W Z Score'] = [np.nan]*tdf4.shape[0]
    
    df_2019 = pd.concat([tdf2, tdf3, tdf4], axis=0)

del tdf2
del tdf3

display_df = df_2019.copy(deep=True)
items = ['file_year', 'HAI Measures End Date', 'HAI Measures Start Date',
         'CAUTI Footnote', 'CAUTI W Z Score', 'CAUTI derived W Z Score', 'CAUTI derived Winsorized SIR',
         'CDI Footnote', 'CDI W Z Score', 'CDI derived W Z Score',
         'CLABSI Footnote', 'CLABSI W Z Score', 'CLABSI derived W Z Score',
         'MRSA Footnote', 'MRSA W Z Score', 'MRSA derived W Z Score',
         #'Total HAC Footnote', 'Total HAC Score',
         ]


display_df = display_df.filter(items=items)
#display_df = display_df[display_df['CAUTI W Z Score'].isin([np.nan, float("NaN")])]
#display_df.sort_values(by='CAUTI derived W Z Score', inplace=True, ascending=False)
display_df.head(20)

,file_year,CAUTI Footnote,CAUTI W Z Score,CAUTI derived W Z Score,CAUTI derived Winsorized SIR,CDI Footnote,CDI W Z Score,CDI derived W Z Score,CLABSI Footnote,CLABSI W Z Score,CLABSI derived W Z Score,MRSA Footnote,MRSA W Z Score,MRSA derived W Z Score
0,2019,NaN,0.2434,0.246201,1.0107,NaN,-0.8858,-0.886730,NaN,0.4992,0.504848,NaN,0.5608,0.561932
1,2019,NaN,0.0201,0.023168,0.8832,NaN,-1.0295,-1.032210,NaN,-0.3761,-0.376008,NaN,0.8634,0.864312
2,2019,NaN,0.6987,0.702936,1.2718,NaN,-0.9273,-0.928850,NaN,0.4309,0.436005,NaN,1.1477,1.148127
5,2019,NaN,0.2260,0.229932,1.0014,NaN,-0.8085,-0.807755,NaN,0.8445,0.851847,NaN,-0.6832,-0.683209
6,2019,NaN,-1.5202,-1.521791,0.0000,NaN,-1.2064,-1.207894,NaN,-1.4878,-1.492526,NaN,-1.4406,-1.440162
7,2019,NaN,-0.3706,-0.369544,0.6587,NaN,-0.3692,-0.368546,NaN,-0.0400,-0.038659,NaN,-0.6180,-0.617984
9,2019,NaN,-0.6306,-0.629486,0.5101,NaN,-1.0129,-1.014475,NaN,1.6940,1.705425,NaN,1.4804,1.481113
12,2019,NaN,0.1510,0.153664,0.9578,NaN,0.9598,0.964047,NaN,-0.1305,-0.129212,NaN,0.0625,0.063039
13,2019,NaN,0.6080,0.613023,1.2204,NaN,1.3356,1.340631,NaN,1.7051,1.715445,NaN,0.2899,0.291329
14,2019,NaN,0.8034,0.808242,1.3320,NaN,-0.5543,-0.555037,NaN,0.4475,0.452149,NaN,0.8534,0.854779


In [14]:

features = ['CAUTI derived SIR', 'CAUTI W Z Score', 'CAUTI derived W Z Score',
            'CDI derived SIR', 'CDI W Z Score', 'CDI derived W Z Score',
            'CLABSI derived SIR', 'CLABSI W Z Score', 'CLABSI derived W Z Score',
            'MRSA derived SIR', 'MRSA W Z Score', 'MRSA derived W Z Score', 
            'SSI W Z Score', 'PSI-90 W Z Score', 'Total HAC Score']

for f in features:
    df_2019[f] = df_2019[f].astype(str)
    df_2019[f] = pd.to_numeric(df_2019[f], errors='coerce')
    

for hai in hais:
    print(hai)
    
    ls1 = df_2019[hai + ' W Z Score'].tolist()
    ls2 = df_2019[hai + ' derived W Z Score'].tolist()
    final_ls = []
    
    for i, val in enumerate(ls1):
        if np.isnan(val) == False and np.isnan(ls2[i]) == True:
            final_ls.append(ls2[i])
        elif np.isnan(val) == False and np.isnan(ls2[i]) == False:
            final_ls.append(ls2[i])
        elif np.isnan(val) == True and np.isnan(ls2[i]) == False:
            final_ls.append(ls2[i])
        elif np.isnan(val) == True and np.isnan(ls2[i]) == True:
            final_ls.append(ls2[i])
        else:
            final_ls.append(ls2[i])
            
    df_2019[hai + ' derived W Z Score'] = final_ls


CAUTI
CLABSI
MRSA
CDI


## Attempt to reproduce HAC scores for 2019

In [15]:
print('Results from attempting to reproduce Yes/No penalty assignments:\n')
print('Excluded from results below:')
print('1. MD hospitals')
print('2. Hospitals with payment reduction values other than Yes or No\n')

df_2019.dropna(how='all', axis=1, inplace=True)

holdout_df = df_2019[(df_2019['State'] == 'MD') | ~df_2019['Payment Reduction'].isin(['Yes', 'No']) | (df_2019['Total HAC Score'].isin([float("NaN"), np.nan]))]
df_2019 = df_2019[(df_2019['State'] != 'MD') & (df_2019['Payment Reduction'].isin(['Yes', 'No'])) & (~df_2019['Total HAC Score'].isin([float("NaN"), np.nan]))]

hac_scores = []
ct1 = 0
ct2 = 0


for hosp in df_2019['Facility ID'].tolist():
    tdf = df_2019[df_2019['Facility ID'] == hosp]

    d1 = 0
    d2 = 0
        
    w_ls = []
    sum_ls = []

    # Use original scores for all HAIs to test whether can penalties be reproduced when 
    # using data from the HACRP files
    #m_ls = ['CDI W Z Score', 'CAUTI W Z Score', 'CLABSI W Z Score', 'MRSA W Z Score', 'SSI W Z Score']

    # Use original SSI scores but derived scores for CDI, CAUTI, CLABSI, and MRSA for actual results
    m_ls = ['CDI derived W Z Score', 'CAUTI derived W Z Score', 'CLABSI derived W Z Score', 'MRSA derived W Z Score', 'SSI W Z Score']
                
    s = 0
    w = 0
    for m in m_ls:
        v = tdf[m].tolist()
        if len(list(set(v))) > 1:
            print('len(list(set(v))) > 1')
            sys.exit()

        v = tdf[m].iloc[0]

        if np.isnan(v) == False: 
            s += v
            w += 1

    if w == 0:
        d2 = np.nan
    else:
        d2 = s/w
        
    d1 = float(tdf['PSI-90 W Z Score'].iloc[0])
    d2_o = float(tdf['Domain 2 Score'].iloc[0])
    state = tdf['State'].iloc[0]

    if np.isnan(d1) == True: 
        # if no score for Domain 1, then total HAC score will be based entirely on Domain 2
        if np.isnan(d2) == True and np.isnan(d2_o) == True:
            hac_scores.append(d2)

        elif np.isnan(d2) == True and np.isnan(d2_o) == False:
            hac_scores.append(d2_o)

        elif np.isnan(d2) == False and np.isnan(d2_o) == False:
            hac_scores.append(d2)

        elif np.isnan(d2) == False and np.isnan(d2_o) == True:
            hac_scores.append(d2_o)

    elif np.isnan(d1) == False:
        # if there is a score for Domain 1 ...

        if state == 'MD':
            # states in MD should not have scores for domain 1
            print('Error:')
            print('Domain 1:', d1)
            print('State:', state)
            print('Hospitals in this state should not have domain 1 scores.\n')
            
        # Domain 2:

        # If the derived score is NaN and the original score is NaN ...
        if np.isnan(d2) == True and np.isnan(d2_o) == True:
            hac_scores.append(d1)
        
        # If the derived score is a float but the original score is NaN ...
        elif np.isnan(d2) == False and np.isnan(d2_o) == True:
            hac_scores.append(d1)
            
        # If the derived score is NaN but the original score is a float ...
        elif np.isnan(d2) == True and np.isnan(d2_o) == False:
            ct1 += 1
            hac_scores.append(0.15*d1 + 0.85*d2_o)

        # If the derived score is a float and the original score is a float ...
        elif np.isnan(d2) == False and np.isnan(d2_o) == False:
            if d2 != d2_o:
                ct2 += 1
                #print(d2, d2_o)
            hac_scores.append(0.15*d1 + 0.85*d2)

    else:
        print(d1, ',', d2, ',', d2_o)
        break

        
df_2019['Total HAC Score (derived)'] = hac_scores
print(df_2019.shape[0], 'hospitals in hac_df')


Results from attempting to reproduce Yes/No penalty assignments:

Excluded from results below:
1. MD hospitals
2. Hospitals with payment reduction values other than Yes or No

3204 hospitals in hac_df


In [16]:
p75 = np.nanpercentile(df_2019['Total HAC Score'], 75)
p75 = 0.3429
print('p75:', p75)

pr = []
for hosp in df_2019['Facility ID'].tolist():
    tdf = df_2019[df_2019['Facility ID'] == hosp]

    score = tdf['Total HAC Score'].iloc[0]

    if np.isnan(score) == True:
        pr.append('No')
    elif score <= p75:
        pr.append('No')
    elif score > p75:
        pr.append('Yes')
    else:
        print('This score is an error:', score)
        sys.exit()

df_2019['Payment Reduction (derived)'] = pr
    
o_list = df_2019['Payment Reduction'].tolist()
d_list = df_2019['Payment Reduction (derived)'].tolist()

same = 0
diff = 0
res_ls = []
for i, o in enumerate(o_list):
    if o == d_list[i]:
        same += 1
        res_ls.append(1)
    else:
        diff += 1
        res_ls.append(0)
            
df_2019['Payment Reduction Reproduced?'] = res_ls
    
print(same, "Penalty assignments were reproduced")
print(diff, "Penalty assignments were not reproduced")
print(str(np.round(100 * same/(same+diff), 2)) + '% penalty assignments were reproduced\n')
    
df_2019['HAC delta'] = df_2019['Total HAC Score'] - df_2019['Total HAC Score (derived)']
df_2019['CDI delta'] = df_2019['CDI W Z Score'] - df_2019['CDI derived W Z Score']
df_2019['MRSA delta'] = df_2019['MRSA W Z Score'] - df_2019['MRSA derived W Z Score']
df_2019['CAUTI delta'] = df_2019['CAUTI W Z Score'] - df_2019['CAUTI derived W Z Score']
df_2019['CLABSI delta'] = df_2019['CLABSI W Z Score'] - df_2019['CLABSI derived W Z Score']


p75: 0.3429
3204 Penalty assignments were reproduced
0 Penalty assignments were not reproduced
100.0% penalty assignments were reproduced



In [17]:
ls1 = list(df_2019)
ls2 = list(holdout_df)
ls = list(filter(lambda x:x in ls1, ls2))
print(df_2019.shape)
print(holdout_df.shape)
df_2019 = df_2019.merge(holdout_df, how='outer', on=ls)
print(df_2019.shape)


(3204, 63)
(70, 55)
(3274, 63)


In [18]:
tdf1 = df_2019[df_2019['Payment Reduction'] == 'Yes']
tdf2 = df_2019[df_2019['Payment Reduction'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_2019[df_2019['Payment Reduction'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])

print(df_2019.shape[0])
print(len(df_2019['Facility ID'].unique()))

df_2019.to_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/HAI_HAC_2019.pkl', protocol=5)


0.2479082739386427
0.7520917260613573
3274
3274
